In [5]:
#!/usr/bin/env python3
import os
from numpy import *
from matplotlib.pyplot import *
matplotlib.use("Agg")
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
Writer=animation.writers['ffmpeg']
writer=Writer(bitrate=1800)
R=3.2
S=28.574
P=7200
C=510
c=840
p=2600
k=2.835
a=k/(c*p)
b=2*k*pi*R/(C*P*S)
rate=[]
time=0
t1=0
t2=0
def runheat(x, u, End, fixed=False, s=10, n=1500, tstep=0.0001, sample=400):
    global R, S, P, C, c, p, k, rate, time, t1, t2
    step=(s-R)/n
    r=arange(R, s, step)[0:n]
    tstep*=3600
    tsample=tstep*sample
    t=0
    dT=0
    if not fixed:
        while u[0]>End:
            for i in range(sample):
                u[0]+=b*(u[1]-u[0])/step*tstep
                d1=(u[2:n+2]-u[0:n])/(2*step)
                d2=(u[2:n+2]+u[0:n]-2*u[1:n+1])/(step**2)
                u[1:n+1]+=a*tstep*(d1/r+d2)
            t+=1
            print(u[0])
            yield u[::30]
        print("End time: %.2f"%(t*tsample/3600))
    else:
        while dT<End:
            for i in range(sample):
                dT+=-b*(u[1]-u[0])/step*tstep
                d1=(u[2:n+2]-u[0:n])/(2*step)
                d2=(u[2:n+2]+u[0:n]-2*u[1:n+1])/(step**2)
                u[1:n+1]+=a*tstep*(d1/r+d2)
            t+=1
            print(dT)
            rate.append(dT/End)
        yield u[::30]
def samplex(s=10, n=1500):
    return arange(n+2)*(s-3.2)/n+3.2
# Cache data for data analysing
def cachedata():
    if not os.path.isfile("data.npy"):
        print("Caching data to data.npy...")
        np.save("data.npy", array([i.copy() for i in runheat()]))
        np.save("time.npy", time)
        np.save("rate.npy", array(rate))
def gentrace():
    #cachedata()
    heattrace=np.load("data.npy")
    time=np.load("time.npy")/3600
    rate=np.load("rate.npy")
    print(shape(heattrace))
    l=shape(heattrace)[0]
    heattrace=array(heattrace)
    x=samplex()[::30]
def drawxy(x, y):
    fg,ax=subplots()
    ln,=ax.plot([],[],'-',lw=2)
    ax.set_xlim(3, 10)
    ax.set_ylim(0, 1500)
    ax.set_xlabel("r/m")
    ax.set_ylabel("T/C")
    grid()
    ttime = ax.text(0.6, 0.90, '', transform=ax.transAxes)
    tsolid = ax.text(0.6, 0.85, '', transform=ax.transAxes)
    def run(i):
        ln.set_data(x,heattrace[i])
        ttime.set_text('Time = %.1fh' % time[i])
        tsolid.set_text('Ratio = %.2f' % rate[i])
        return ln, ttime, tsolid
    ani=FuncAnimation(fg, run, range(0, k,10), blit=True, interval=20, repeat=True)
    ani.save("cooling.mp4", writer=writer)

#ani=FuncAnimation(fg, run, range(k-200, l, 60), blit=True, interval=20, repeat=True)
#ani.save("transition.mp4", writer=writer)
#show()